In [1]:
# dictionary -> string(32chars) and jsonObject(16KB)

import os
import sys
import json
import datetime

class Util:
    def parse(d): 
        dictionary = dict() 
        # Removes curly braces and splits the pairs into a list 
        pairs = d.strip('{}').split(', ') 
        for i in pairs: 
            pair = i.split(': ') 
            # Other symbols from the key-value pair should be stripped. 
            dictionary[pair[0].strip('\'\'\"\"')] = pair[1].strip('\'\'\"\"') 
        return dictionary

    def contains(path, key): 
        try: 
            file = open(path, 'rt') 
            lines = file.read().split('\n') 
            for l in lines: 
                if l != '': 
                    dictionary = parse(l) 
                    #print(dictionary.keys) 
                    if list(dictionary.keys())[0] == key:
                        return True
            file.close() 
        except: 
            print("Something unexpected occurred!")

        return False
        
    def secTillNow():
        now = datetime.datetime.now()
        midnight = datetime.datetime.combine(now.date(), datetime.time())
        seconds = (now - midnight).seconds
        return seconds

class Library:
    
    '''Describe this class'''
    
    def __init__(self, path = None):
        if path == None:
            self.path = os.getcwd()
        else:
            self.path = path
        
        self.__fileName = "library.txt"  #library file
        
        try:
            self.path = os.path.join(self.path, self.__fileName)
            with open(self.path, "a+") as file:
                file.close()
                
        except FileNotFoundError as fnf_error:
            print("Error : ", fnf_error)
            
        except:
            print('Unable to initialize library')
            
            
    def create(self, key, value, timeToLive = 86400):
        
        if Util.contains(self.path, key) == False:            
            
            fileStats = os.stat(self.path)
            fileSize = (fileStats.st_size)/(1024*1024)
            if fileSize >= 1024:
                print("file size is too large")
                return "Operation Unsuccessfull"
            #else:
                #print(fileSize)
            dict = {} 
            
            if type(key) == type("") and len(key) <= 32:
                if key.isalpha():
                    try: 
                        value = json.loads(value) 
                        # print ("Is valid json? true") 
                    except ValueError as e: 
                        print ("It is not valid json object")
                        return
                    if(sys.getsizeof(value) <= (1024*16)):
                        dict[key] = value
                        tSec = Util.secTillNow()
                        if timeToLive == 86400:
                            dict['timeToLive'] = timeToLive-1
                        else:
                            dict['timeToLive'] = (tSec + timeToLive)
                        try:
                            with open(self.path, "a+") as file:
                                file.write(str(dict)+"\n")
                                file.close()
                                print("The value was appended in library")
                        except:
                            print("Unable to perform create operation")
                    else:
                        print("json object size exceeded")
                else:
                    print("key should be alpha numeric")

            else:
                print("Key size exceeded")
            
        else:
            print("Key is already present")
            
    
    def read(self, key):
        try: 
            file = open(self.path, 'rt') 
            lines = file.read().split('\n') 
            for l in lines: 
                if l != '': 
                    dictionary = eval(l)
                    #print(dictionary.keys) 
                    if list(dictionary.keys())[0] == key:
                        if dictionary['timeToLive'] >= Util.secTillNow():
                            return dictionary[key]
                        else:
                            return "json object is not accessible"
            file.close() 
        except: 
            print("Something unexpected occurred!")

        return "Key Not Found"
    
    def delete(self, key):
        try:
            flag = False
            readFile = open(self.path, "r")
            lines = readFile.readlines()
            readFile.close()
            #print(lines)
            
            writeFile = open(self.path, "w")
            for line in lines:
                dictionary = eval(line.strip("\n"))
                if list(dictionary.keys())[0] == key:
                    if dictionary['timeToLive'] < Util.secTillNow():
                        writeFile.write(line)
                    else:
                        flag = True
                        deletedVal = dictionary
                else:
                    writeFile.write(line)
            writeFile.close()
            
            if flag:
                print("Value was deleted")
                return deletedVal
            else:
                print("Either the key is not present or key is unaccessible")
                
        except:
            print("Something unexpected occured!")

In [2]:
class Test:
    '''Testing Library class'''
    
    #l1 = Library()
    l1 = Library("D:")
    #l3 = Library("xyz")
    
    
    #l1.create("abc", '{"name": "Bob", "languages": "English"}', 800)
    #l1.create("ab", '{"name": "Bob", "languages": "English"}', 800)
    #l1.create("abc1", '{"name": "Bob", "languages": "English"}', 800)
    #l1.create("a", '{"name": "Bob "languages": "English"}', 800)
    #l1.create("abcd", '{"name": "Job", "languages": "English"}', 800)
    #l1.create("abcdf", '{"name": "Job", "languages": "English"}')
    #print(l1.read("abcd"))
    #print(l1.read("abcde"))
    #print(Util.secTillNow())
    
    l1.delete("abcf")
    l1.create("abcf", '{"name": "cob", "languages": "English"}', 800)

Either the key is not present or key is unaccessible
The value was appended in library
